In [30]:
import numpy as np 
import pandas 
import xgboost as xgb
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import accuracy_score # This seems to only work for classification
from sklearn.metrics import explained_variance_score
import math


# Plot parameters 
import matplotlib
import matplotlib.pyplot as plt

font = {'family' : 'Tahoma',
        'weight' : 'normal',
        'size'   : 24}

matplotlib.rc('font', **font)

In [31]:
# These are the trials that we have good data for 
acceptable_trials = ['m07_t01_15', 'm07_t03_15', 'm07_t06_15','m10_t02_16','m11_t02_16','m11_t04_16',
                   'm12_t02_16','m14_t05_16', 'm14_t03_16', 'm15_t01_16', 'm15_t03_16']



In [32]:

# Try some other hyperparameters
# Hyperparameters to build the xGBoost regressor model
obj ='reg:linear'
csbt = .15 
lr = .01
md = 3
n_est = 750
ss = .7
alp = 1



In [33]:
# I used this framework to determine if I could create predictions that were as good with fewer data columns.
# My conclusion is that using all of these gives better predictions.

exp_var = pandas.DataFrame((np.zeros((len(acceptable_trials),5)))*np.nan, index = [np.arange(0,len(acceptable_trials))], columns = ['Explained_variance', 'rms', 'mse', 'mse_with_mean', 'Number_estimators'])

column_sets = [['M6_c', 'A59_c', 'M3_c', 'I20_I10', 'peaks']]

diffcolumns_exp_var = pandas.DataFrame((np.zeros((len(column_sets),3)))*np.nan, index = [np.arange(0,len(column_sets))], columns = ['Explained_variance', 'mse', 'mse_with_mean'])
print('got it')
c_count = 0 
for c in column_sets:
    count = 0
    exp_var = pandas.DataFrame((np.zeros((len(acceptable_trials),3)))*np.nan, index = [np.arange(0,len(acceptable_trials))], columns = ['Explained_variance', 'mse', 'mse_with_mean'])
    for trial in acceptable_trials:
        model_mse = 1
        meanData_mse = 0
        n_est = 500
        #while model_mse > meanData_mse:
        # Build the regressor object with optional kwargs -- this will be the same throughout. 
        xg_reg = xgb.XGBRegressor(objective =obj, colsample_bytree = csbt, learning_rate = lr,
                        max_depth = md, n_estimators = n_est, subsample = ss, alpha = alp)

        # Import data, interpolate nans and subset to desired columns
        d = pandas.read_csv('../DataProcessing/ProcessedData/' + trial + '_det.csv')
        d = d.interpolate()
        d = d[['D10', 'M6_c', 'I20_I10', 'M3_i', 'M3_c', 'A59_c', 'peaks','seconds']]

            # Drop any rows that have a nan
            #d = d.dropna(how = 'any').reset_index(drop = 'True')

            # Add time history in the form of time shifted columns
            # I chose to  do this in a recursive style, referencing the j-1 shift and shifting it further by 1. It could be done without the recursion.
        columns = c#column_sets[c] # ['M6_c', 'I20_I10'] # 'M3_i','A59_c','M6_c',
        shift_column_names = []
        for col in columns:
            d[col + '_' + str(0)] = d[col].shift(-1)
            shift_column_names.append(col + '_' + str(0))

        for j in np.arange(1,6):
            for col in columns:
                d[col + '_' + str(j)] = d[col + '_' + str(j-1)].shift(-1)
                shift_column_names.append(col + '_' + str(j))

        # This dropna will actually cut the total length of the dataframe since by time shifting there will be some columns that are nearly a whole ISI shorter than the original data length      
        d = d.dropna()

        # Subset the data into predictors (X) and predicted (y)
        modeled_data = 'D10'
        X = (d[columns + shift_column_names+ ['seconds']])
        y = (d[[modeled_data]])

        # Split the data into test and train
        test_percent = .25 # Percentage of data to be reserved for test set. This needs to be kept as a chunk with time dependence
        length_test_set = math.ceil(test_percent*len(d)) # Round up

        test_X = X[:length_test_set]
        test_y = y[:length_test_set]

        train_X = X[length_test_set:]
        train_y = y[length_test_set:]

        test_y_mean = np.full((len(test_y)), np.mean(test_y))

        # Fit the model
        xg_reg.fit(train_X, train_y)

        # Model prediction of the training set
        y_pred = xg_reg.predict(train_X)

        plt.rcParams['figure.figsize'] = [14, 6]   

        # Model prediction of the test set
        y_pred = xg_reg.predict(test_X)
        exp_var.Explained_variance.iloc[count] = round(explained_variance_score(y_pred,test_y), 2)
        #exp_var.rms.iloc[count] = round(((sum((np.concatenate(np.array(test_y)) - y_pred)**(2)))/len(y_pred))**(1/2), 3)
        exp_var.mse.iloc[count] = round(mean_squared_error(y_pred, test_y), 2)
        exp_var.mse_with_mean.iloc[count] = round(mean_squared_error(test_y_mean, test_y),2)
        #exp_var.Number_estimators.iloc[count] = n_est
        model_mse = round(mean_squared_error(y_pred, test_y), 2)
        meanData_mse = round(mean_squared_error(test_y_mean, test_y),2)
              
        count += 1
            
    diffcolumns_exp_var.mse.iloc[c_count] = exp_var.mse.mean()
    diffcolumns_exp_var.Explained_variance.iloc[c_count] = exp_var.Explained_variance.mean()
    diffcolumns_exp_var.mse_with_mean.iloc[c_count] = exp_var.mse_with_mean.mean()
        
    c_count += 1





got it
[16:48:00] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[16:48:00] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[16:48:00] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[16:48:00] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[16:48:01] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[16:48:01] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[16:48:01] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[16:48:01] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[16:48:01] WARNING: src/objective/regression_obj.cu:152: reg:line

In [36]:
diffcolumns_exp_var

,Explained_variance,mse,mse_with_mean
0,-4.309091,0.263636,0.283636


In [38]:
# I want to do the same thing in essence -- what dat are needed to create a good prediction
# except I will do this on the train on n-1 trials and test on the nth. 

# Split data into test and train sets
acceptable_trials = ['m07_t01_15', 'm07_t03_15', 'm07_t06_15','m10_t02_16','m11_t02_16','m11_t04_16',
                   'm12_t02_16','m14_t05_16', 'm14_t03_16', 'm15_t01_16', 'm15_t03_16']

# Split the data into test and train sets
test_trials = []
train_trials = []
for i in np.arange(0,len(acceptable_trials)):
    test_trials.append(acceptable_trials[i])
    train_trials.append((acceptable_trials[:(i)] + acceptable_trials[(i+1):]))
    
 


In [49]:
# train on N-1 of N trials and test on the removed trial
column_sets = [['I20_I10'], ['M6_c', 'I20_I10', 'peaks']]
test_column = 'D10'
#predictor_columns = ['Wingbeat_freq', 'M6_c', 'I20_I10', 'M3_i', 'A59_c']
#exp_var_all_trials = pandas.DataFrame((np.zeros((len(acceptable_trials),3)))*np.nan, index = [np.arange(0,len(acceptable_trials))], columns = ['Explained_variance', 'mse', 'mse_with_mean'])
exp_var_all_trials_all_col_sets = pandas.DataFrame((np.zeros((len(column_sets),3)))*np.nan, index = [np.arange(0,len(column_sets))], columns = ['Explained_variance', 'mse', 'mse_with_mean'])

n_est = 1850
col_count = 0
for c in column_sets:
    exp_var_all_trials = pandas.DataFrame((np.zeros((len(acceptable_trials),3)))*np.nan, index = [np.arange(0,len(acceptable_trials))], columns = ['Explained_variance', 'mse', 'mse_with_mean'])
    for i in np.arange(0,len(train_trials)): 
        xg_reg = xgb.XGBRegressor(objective =obj, colsample_bytree = csbt, learning_rate = lr,
                    max_depth = md, n_estimators = n_est, subsample = ss, alpha = alp)


        # Prep the data

        # Base case
        trial = train_trials[i][0]
        d = pandas.read_csv('../DataProcessing/ProcessedData/' + trial + '_det.csv')
        #d = d[['D10', 'D11', 'D20', 'I20I11_I10', 'I20_I10',
        #       'fitting_error', 'seconds', 'tif_num', 'M3_c', 'M6_c', 'A51_c', 'A59_c',
        #       'M3_i', 'M6_i', 'A59_i', 'A51_i', 'Sum', 'peaks', 'ISI']]

        d = d.interpolate() # Fill any nan values
        d = d[['D10', 'I20_I10', 'M3_i', 'A59_c', 'M6_c', 'peaks']]
        #d = d.dropna(how = 'any').reset_index(drop = 'True')

        columns = c#['M3_i', 'M6_c', 'I20_I10', 'A59_c']
        shift_column_names = []
        for col in columns:
            d[col + '_' + str(0)] = d[col].shift(-1)
            shift_column_names.append(col + '_' + str(0))

        for j in np.arange(1,11):
            for col in columns:
                d[col + '_' + str(j)] = d[col + '_' + str(j - 1)].shift(-1)
                shift_column_names.append(col + '_' + str(j))

        d = d.dropna()

        # Subset the data into predictors (X) and predicted (y)
        X = (d[columns + shift_column_names])
        y = (d[['D10']])


        # Subsequent cases

        # Create training set
        for trial in train_trials[i][1:]: 
            d = pandas.read_csv('../DataProcessing/ProcessedData/' + trial + '_det.csv')
            d = d[['D10', 'D11', 'D20', 'I20I11_I10', 'I20_I10',
                   'fitting_error', 'seconds', 'tif_num', 'M3_c', 'M6_c', 'A51_c', 'A59_c',
                   'M3_i', 'M6_i', 'A59_i', 'A51_i', 'Sum', 'peaks', 'ISI']]

            d = d.interpolate()
            d = d[['D10', 'I20_I10', 'M3_i', 'A59_c', 'M6_c', 'peaks']]
            #d = d.dropna(how = 'any').reset_index(drop = 'True')

            for col in columns:
                d[col + '_' + str(0)] = d[col].shift(-1)
                #shift_column_names.append(col + '_' + str(0))

            for j in np.arange(1,11):
                for col in columns:
                    d[col + '_' + str(j)] = d[col + '_' + str(j - 1)].shift(-1)

            d = d.dropna()

            # Subset the data into predictors (X) and predicted (y)
            X = X.append(d[columns + shift_column_names])
            y = y.append(d[['D10']])

        X = X.reset_index(drop = 'True')
        y = y.reset_index(drop = 'True')

        xg_reg.fit(X, y)
        y_pred = xg_reg.predict(X)

        # Use the last df as a test set
        trial = test_trials[i]
        d = pandas.read_csv('../DataProcessing/ProcessedData/' + trial + '_det.csv')
        #d = d[['D10', 'D11', 'D20', 'I20I11_I10', 'I20_I10',
        #       'fitting_error', 'seconds', 'tif_num', 'M3_c', 'M6_c', 'A51_c', 'A59_c',
        #       'M3_i', 'M6_i', 'A59_i', 'A51_i', 'Sum', 'peaks', 'ISI']]
        d = d.interpolate()
        d = d[['D10', 'I20_I10', 'M3_i', 'A59_c', 'M6_c', 'peaks']]
        d = d.dropna(how = 'any').reset_index(drop = 'True')

        # Create time shifted predictor data
        for col in columns:
            d[col + '_' + str(0)] = d[col].shift(-1)
        for j in np.arange(1,11):
            for col in columns:
                d[col + '_' + str(j)] = d[col + '_' + str(j - 1)].shift(-1)
        d = d.dropna()

        X = (d[columns + shift_column_names])
        y = (d[['D10']])
        y_mean = np.full((len(y)), np.mean(y))


        # Use the trained regressor to predict y from the test set X
        y_pred = xg_reg.predict(X)

        # Calculate and store error metrics (explained variance, a manual root mean square and mean square error from scikitlearn)
        exp_var_all_trials.Explained_variance.iloc[i] = explained_variance_score(y_pred,y)
        exp_var_all_trials.mse.iloc[i] = mean_squared_error(y_pred,y)
        exp_var_all_trials.mse_with_mean.iloc[i] = mean_squared_error(y_mean, y)
        
        #plt.rcParams['figure.figsize'] = [14, 6]
        #fig = plt.figure()
        #ax = fig.add_subplot(111)
        #ax.plot((np.arange(0,len(y))*(1/200))[0:50], y[0:50], c = 'black', label = 'Target')
        #ax.plot((np.arange(0,len(y))*(1/200))[0:50], y_mean[0:50], c = 'black', alpha = .5, label = 'Target mean')
        #ax.scatter((np.arange(0,len(y))*(1/200))[0:50], y_pred[0:50], c = 'black', marker = 'x', label = 'Prediction')
        #ax.legend(loc = 'upper right')
        #ax.set_title('Prediction of lattice spacing change in time')
        #ax.set_ylabel('Lattice spacing (percent change)')#modeled_data)
        #ax.set_xlabel('Time (s)')
        #ax.text(0.02,.9, 'Explained variance score: ' + str(round(explained_variance_score(y_pred,y), 2)), family="sans-serif", fontsize = 20, transform=ax.transAxes) # 0.03, 2.,
        #ax.yaxis.set_major_locator(plt.MaxNLocator(4)) 
        #ax.xaxis.set_major_locator(plt.MaxNLocator(5))
        #plt.savefig('/Users/sagemalingen/Desktop/RandomForest_Figs/Predicted_AllTrialTrain_' + test_trials[i] + '.png', dpi = 400)
        #plt.show()
    exp_var_all_trials_all_col_sets.mse.iloc[col_count] = exp_var_all_trials.mse.mean()
    exp_var_all_trials_all_col_sets.mse_with_mean.iloc[col_count] = exp_var_all_trials.mse_with_mean.mean()
    exp_var_all_trials_all_col_sets.Explained_variance.iloc[col_count] = exp_var_all_trials.Explained_variance.mean()
    col_count += 1


[17:37:11] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[17:37:13] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[17:37:15] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[17:37:17] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[17:37:20] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[17:37:22] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[17:37:24] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[17:37:26] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[17:37:29] WARNING: src/objective/regression_obj.cu:152: reg:linear is n

In [44]:
exp_var_all_trials_all_col_sets

,Explained_variance,mse,mse_with_mean
0,-187.308610,1.181208,0.270265
1,-7.530685,0.812408,0.270265
2,-8.481175,0.939539,0.270265
3,-7.287844,0.925092,0.270265


In [ ]:
# Technically is best with only I20_I10, but very similar to having that plus other columns

In [50]:
exp_var_all_trials_all_col_sets

,Explained_variance,mse,mse_with_mean
0,-1.942015,0.698027,0.270265
1,-2.421442,0.915192,0.270265
